# Make nice graph

In [1]:
import sys
sys.path.append('..')
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import json,gzip
import intra_citation as ia 
import GCN_utils as u
%matplotlib inline

2019-05-02 11:58:39,403 - arxivdata.config - WARNING: default output directory is /home/kokeeffe/research/arxiv-public-datasets/analysis/kipf_welling_GCN/arxiv-data


### Profile

In [4]:
import sys
sys.path.append('..')
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import json,gzip,time
import intra_citation as ia 
from arxiv_public_data.oai_metadata import load_metadata
from arxiv_public_data.embeddings.util import load_embeddings, fill_zeros
from GCN_utils import *
%matplotlib inline

N = 100

#Load graph & metadata
t1 = time.time()

#Meta data -- need this later
dirname = '/home/kokeeffe/research/arxiv-public-datasets/arxiv-data'
m = load_metadata(dirname + '/oai-arxiv-metadata-2019-03-01.json.gz')

graph_saved = False    #for debugging, I've already made the full graph
if graph_saved == True:
    dirname = 'data'
    G = nx.read_gpickle(dirname + '/full_graph.gpickle')
else:
    dirname = '/home/kokeeffe/research/arxiv-public-datasets/arxiv-data/output'
    fname = dirname + '/internal-citations.json.gz'
    q = json.load(gzip.open(fname, 'rt', encoding='utf-8'))
    G = ia.makegraph(q)
    G = sync_G_with_metadata(G,m)   #G is missing some articles (see func for more details)

#Select subgraph if specified
if N != 0:
    comps = nx.weakly_connected_components(G)
    biggest = max(comps, key=len)
    G_cc = G.subgraph(biggest)
    nodes = list(G_cc.nodes())[:N]
    G_sub = G_cc.subgraph(nodes)
else:
    G_sub = G

nodes_string = np.array(G_sub.nodes())            #nodes labeled by strings
nodes_int = np.array(range(G_sub.number_of_nodes()))       #nodes labeled by ints
t2 = time.time()
print('Loading graph took ' + str((t2-t1)/60.0) + ' mins')

(# Nodes in G, # articles in meta-data) = (1506500, 1506500)
So the syncing worked
Loading graph took 1.5179288665453592 mins


In [41]:
#Load features
t1 = time.time()
dirname = '/home/kokeeffe/research/arxiv-public-datasets/arxiv-data/output/embeddings'
title_vecs = load_titles(G, m, nodes_string, dirname)
abstract_vecs = load_abstracts(G, m, nodes_string, dirname)
fulltext_vecs = load_fulltext(G, m, nodes_string, dirname)

#Load labels
dirname = '/home/kokeeffe/research/arxiv-public-datasets/arxiv-data'
labels_cat = load_labels(nodes_string, m)
t2 = time.time()
print( 'Loading features & labels took ' + str((t2-t1)/60.0) + ' mins')

Loading graph took 2.0024171551068624 mins
